# Análise de Tópicos

#### Importação de Bibliotecas

In [1]:
import pandas as pd
import spacy
from bertopic import BERTopic

import re
from datetime import datetime

from gensim.utils import simple_preprocess

import nltk
nltk.download(["stopwords", "rslp"])
stopwords = nltk.corpus.stopwords.words("portuguese")

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


#### Leitura do arquivo

In [2]:
df_reviews = pd.read_csv("guacamole_reviews.csv")
df_reviews.head()

,Date,Review
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já..."
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest..."
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó..."


In [3]:
df_comments = df_reviews["Review"]
df_dates = df_reviews["Date"]

#### Limpeza dos dados

In [4]:
def remove_stopwords(text):
    words = simple_preprocess(text)
    phrase_adjusted = " ".join([word for word in words if word not in stopwords])
    return phrase_adjusted.lower()

spacy_lemma = spacy.load("pt_core_news_sm")

def lemmatizer(text, postags_permit=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = spacy_lemma(text.lower())
    doc_lemma = " ".join([token.lemma_ for token in doc if token.pos_ in postags_permit])
    return doc_lemma

In [5]:
def parse_date(date_str):
    months = {
        'janeiro': 'January', 'fevereiro': 'February', 'março': 'March', 'abril': 'April',
        'maio': 'May', 'junho': 'June', 'julho': 'July', 'agosto': 'August',
        'setembro': 'September', 'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'
    }
    match = re.match(r'(\d+) de (\w+) de (\d+)', date_str)
    if match:
        day, month, year = match.groups()
        month = months[month.lower()]
        return datetime(int(year), list(months.values()).index(month) + 1, int(day)).strftime("%Y-%m-%d")
    else:
        return None

In [6]:
df_reviews["Review Lemma"] = df_reviews["Review"].map(remove_stopwords)
df_reviews["Review Lemma"] = df_reviews["Review"].map(lemmatizer)

In [7]:
df_reviews["Date Formatted"] = df_reviews["Date"].apply(parse_date)

In [8]:
df_reviews.head()

,Date,Review,Review Lemma,Date Formatted
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27


In [9]:
df_comments = df_reviews["Review Lemma"]
doc_comments = df_comments.to_list()
doc_comments[:5]

['atendimento maravilhoso matheu muito atencioso colocar playlist atendente super simpatico',
 'gostar muito prato atendimento solicito comida saboroso certeza voltar mais vez',
 'ameir bom cozinha mexicano já comi espaço muito animar imersivo decoração casa linda atendimento excelente sempre',
 'bom ambiente decoração funcionário prestativo comida boa música agradável divertido mariachi',
 'amoma atendimento matheu bom amar atendimento sensacional comida muito saboroso']

#### BERTopic

In [10]:
topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc_comments)

2023-12-10 20:37:09,348 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/90 [00:00<?, ?it/s]

2023-12-10 20:37:56,325 - BERTopic - Embedding - Completed ✓
2023-12-10 20:37:56,326 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-10 20:38:13,355 - BERTopic - Dimensionality - Completed ✓
2023-12-10 20:38:13,356 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-10 20:38:13,622 - BERTopic - Cluster - Completed ✓
2023-12-10 20:38:13,634 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-10 20:38:13,718 - BERTopic - Representation - Completed ✓


In [11]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,548,-1_muito_bom_atendimento_comida,"[muito, bom, atendimento, comida, lugar, bem, ...",[lugar muito bom agradável comida bom atendime...
1,0,733,0_mexicano_guacamole_restaurante_comida,"[mexicano, guacamole, restaurante, comida, mai...",[comer ótimo preço bom atendimento maravilhoso...
2,1,349,1_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...
3,2,243,2_aniversário_comemorar_amigo_muito,"[aniversário, comemorar, amigo, muito, bom, lu...",[comida maravilhoso ambiente atendimento muito...
4,3,225,3_bebida_drink_cerveja_gelar,"[bebida, drink, cerveja, gelar, bom, atendimen...",[muito animar bom atendimento cerveja gelar co...


In [12]:
topic_model.get_topic(0)

[('mexicano', 0.058143510908718825),
 ('guacamole', 0.032971805584659925),
 ('restaurante', 0.027563850260850974),
 ('comida', 0.02646064982583438),
 ('mais', 0.026437001495451),
 ('bom', 0.023370874521300326),
 ('ter', 0.021428453105078574),
 ('muito', 0.02122114175751319),
 ('bem', 0.01953918390774693),
 ('ambiente', 0.01866330275574135)]

#### Visualização dos tópicos

In [13]:
topic_model.visualize_barchart(top_n_topics=20)

In [14]:
topic_model.visualize_hierarchy()

In [15]:
dates_reviews = df_reviews["Date Formatted"].to_list()

In [16]:
topics_over_time = topic_model.topics_over_time(doc_comments, dates_reviews, datetime_format="%Y-%m-%d", nr_bins=20)

20it [00:00, 53.16it/s]


In [17]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

#### DataFrame com informações completas

In [18]:
df_topics = df_reviews.copy()

df_topics["Topics"] = topics

topic_name = freq.drop(columns=["Count"]).rename(columns={"Topic": "Topics", "Name": "Names"})
df_topics = df_topics.merge(topic_name, how="left")

df_topics.head()

,Date,Review,Review Lemma,Date Formatted,Topics,Names,Representation,Representative_Docs
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04,-1,-1_muito_bom_atendimento_comida,"[muito, bom, atendimento, comida, lugar, bem, ...",[lugar muito bom agradável comida bom atendime...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02,17,17_voltarei_certeza_voltar_voltaremos,"[voltarei, certeza, voltar, voltaremos, delici...",[bom curtir amigo ambiente bem decorado comida...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01,0,0_mexicano_guacamole_restaurante_comida,"[mexicano, guacamole, restaurante, comida, mai...",[comer ótimo preço bom atendimento maravilhoso...
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29,1,1_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27,-1,-1_muito_bom_atendimento_comida,"[muito, bom, atendimento, comida, lugar, bem, ...",[lugar muito bom agradável comida bom atendime...


# Análise de Sentimentos

In [19]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline

model = AutoModelForSequenceClassification.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student')
tokenizer = AutoTokenizer.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student', do_lower_case=False)
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [20]:
sentiment_task("Eu sou feliz")

[{'label': 'positive', 'score': 0.9528123140335083}]

#### Classificação das avaliações e salvamento dos resultados no DataFrame

In [21]:
%%time

df_topics["Sentiment"] = df_topics["Review"].apply(lambda x: sentiment_task(x)[0]["label"])
df_topics["Sentiment Score"] = df_topics["Review"].apply(lambda x: sentiment_task(x)[0]["score"])    

CPU times: total: 29min 9s
Wall time: 4min 53s


In [22]:
df_topics.head()

,Date,Review,Review Lemma,Date Formatted,Topics,Names,Representation,Representative_Docs,Sentiment,Sentiment Score
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04,-1,-1_muito_bom_atendimento_comida,"[muito, bom, atendimento, comida, lugar, bem, ...",[lugar muito bom agradável comida bom atendime...,positive,0.989543
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02,17,17_voltarei_certeza_voltar_voltaremos,"[voltarei, certeza, voltar, voltaremos, delici...",[bom curtir amigo ambiente bem decorado comida...,positive,0.626670
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01,0,0_mexicano_guacamole_restaurante_comida,"[mexicano, guacamole, restaurante, comida, mai...",[comer ótimo preço bom atendimento maravilhoso...,positive,0.967855
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29,1,1_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...,positive,0.940317
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27,-1,-1_muito_bom_atendimento_comida,"[muito, bom, atendimento, comida, lugar, bem, ...",[lugar muito bom agradável comida bom atendime...,positive,0.979101


In [23]:
df_topics.to_csv('guacamole_topics_sentiment.csv', index=False)

In [24]:
count_sentiment = df_topics["Sentiment"].value_counts()
count_sentiment

Sentiment
positive    2648
negative     218
neutral        6
Name: count, dtype: int64